In [1]:
# 为VOC文件夹下的图片提取特征并建立索引
# 1.遍历图片
# 2.提取特征
# 3.建立索引
# 4.存储文件名与索引匹配关系
# 5.保存索引到磁盘

In [2]:
# 导入相关包
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

In [3]:
# 获取所有图片的路径
# image_list =  glob.glob('../VOCdevkit/VOC2012/JPEGImages/*.jpg')
image_list = glob.glob(pathname="../afhq/train/*.jpg")

In [4]:
len(image_list)

16130

In [5]:
# 导入自定义特征提取类
from feature_extract import FeatureExtract
# 实例化
feat = FeatureExtract()
import os
from tqdm.notebook import tqdm

2024-04-30 11:57:45.890170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 11:57:46.026334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/simonwsy/miniconda3/envs/quant/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2024-04-30 11:57:46.026354: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-30 11:57:46.051958: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register 

In [6]:
# 遍历图片，获取特征
# 特征列表
feat_list = []
# 文件名列表
name_list = []
for img_file in tqdm(image_list) :
    # 提取特征
    feat_vec = feat.extractFeat(img_file)
    # 将特征加入列表
    feat_list.append(feat_vec)
    # 解析文件名
    name = img_file.split(os.sep)[-1].split('.')[0]
    # 将文件名加入列表
    name_list.append(name)

    # break 

  0%|          | 0/16130 [00:00<?, ?it/s]

1/1 [==============================] - 0s 64ms/step


In [8]:
# 将文件名列表存入Numpy
name_list = np.array(name_list)
np.save('../afhq/name_list.npy',name_list)

In [10]:
# 将特征存入faiss索引
import faiss

In [11]:
# 创建索引
index = faiss.IndexFlatL2(512)

In [12]:
#feat_list转为numpy数组
feat_list = np.array(feat_list)
# 将特征添加到索引中
index.add(feat_list)

In [13]:
# 查看索引大小
index.ntotal

16130

In [15]:
# 索引文件存入磁盘
faiss.write_index(index, '../afhq/afhq.index')

In [16]:
# 简单测试一下
k = 4
D,I = index.search(feat_list[:4],k)

In [17]:
I

array([[    0,  4044,  8436,  9887],
       [    1, 10735,  4133, 11037],
       [    2, 12626,   838,  5514],
       [    3,  8349,  7151,  6619]])